# LIVRABLE 4

# CARTALLIER Corentin HADDOU Hatim ROUSSEAU Olivier.

Ce projet implémente un **système de transmission et de réception de messages sous forme de signal audio haute fréquence**.
Le message est **converti en binaire**, **encodé avec un code de Hamming** (7,4) pour la correction d’erreurs, puis **modulé en FSK**.
Une fois transmis sous forme de son, le signal peut être reçu, **démodulé**, **décodé** et reconverti en texte.

Le projet se divise en **deux parties** :
1. **Émission** : Transformation du texte en signal audio FSK et enregistrement dans un fichier `.wav`.
2. **Réception** : Analyse du signal, extraction des bits et récupération du message original.

Nous allons explorer le fonctionnement du code en le découpant en étapes explicatives.


# 📡 **Partie 1 : Émetteur (Transmission du Signal FSK)**


## 📌 Importation des Bibliothèques

Les bibliothèques utilisées permettent :
- `numpy` : Manipulation des tableaux numériques.
- `sounddevice` : Lecture du son modulé.
- `scipy.io.wavfile` : Enregistrement du signal en `.wav`.
- `scipy.signal` : Filtrage du signal.
- `os` : Gestion des fichiers et répertoires.


In [ ]:
import numpy as np
import sounddevice as sd
import scipy.io.wavfile as wav
import scipy.signal as sp_signal
import os


## 🎯 Définition des Paramètres

Ces paramètres sont **communs** à l’émetteur et au récepteur pour assurer une transmission correcte :
- `fs` : Fréquence d’échantillonnage du son.
- `bit_rate` : Nombre de bits transmis par seconde.
- `f0` et `f1` : Fréquences associées aux bits `0` et `1`.
- `bit_duration` : Durée de chaque bit.


In [ ]:
fs = 44100  # Fréquence d'échantillonnage (Hz)
bit_rate = 100  # Débit binaire (bits/sec)
f0 = 17000  # Fréquence pour un '0' (Hz)
f1 = 19000  # Fréquence pour un '1' (Hz)
bit_duration = 1 / bit_rate  # Durée d'un bit en secondes


## 📁 Définition du Chemin du Fichier

- **Définit l'emplacement** où sera enregistré le fichier `.wav`.
- **Vérifie** si le dossier existe, et le **crée si nécessaire**.


In [ ]:
# Définition du chemin du fichier de sortie
output_dir = r"C:\Users\CORENTIN CARTALLIER\Desktop\CPI A1\PROJET 3\Avec importation haute fréquence"
filename = os.path.join(output_dir, "fsk_transmission.wav")

# Vérification si le dossier existe, sinon le créer
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


## 🔹 Conversion du Texte en Binaire

Cette fonction transforme chaque caractère ASCII du message en **8 bits**.
🔹 Exemple : `"A"` → `01000001`


In [ ]:
def text_to_bits(text):
    """ Convertit un texte en suite de bits """
    return ''.join(format(ord(c), '08b') for c in text)


## 🔹 Codage de Hamming (7,4)

Le code de Hamming permet **de corriger des erreurs de transmission**.
Il transforme **4 bits de données** en **7 bits**, en ajoutant **3 bits de parité**.


In [ ]:
def hamming_encode(bits):
    """ Encode les bits avec un code de Hamming (7,4) """
    n = len(bits)
    encoded = []

    for i in range(0, n, 4):
        data = bits[i:i+4]
        p1 = int(data[0]) ^ int(data[1]) ^ int(data[2])
        p2 = int(data[0]) ^ int(data[1]) ^ int(data[3])
        p3 = int(data[1]) ^ int(data[2]) ^ int(data[3])
        code = f"{p1}{p2}{data[0]}{p3}{data[1]}{data[2]}{data[3]}"
        encoded.append(code)

    return ''.join(encoded)


## 🔹 Modulation FSK (Frequency Shift Keying)

Cette fonction génère un **signal sonore** :
- `0` → Onde sinusoïdale de **17000 Hz**.
- `1` → Onde sinusoïdale de **19000 Hz**.


In [ ]:
def fsk_modulation(bits):
    """ Génère un signal FSK à partir d'une suite de bits """
    t = np.arange(0, bit_duration, 1/fs)
    signal = np.array([])

    for bit in bits:
        freq = f1 if bit == '1' else f0
        wave = np.sin(2 * np.pi * freq * t)
        signal = np.concatenate((signal, wave))

    return signal


## 🔹 Saisie de l'Utilisateur et Conversion en Binaire

- **Demande à l'utilisateur** d'entrer un message.
- **Convertit** ce message en une séquence de bits.


In [ ]:
# Saisie utilisateur
message = input("Entrez un message à transmettre : ")
bits = text_to_bits(message)
print(f"Message en bits : {bits}")


## 🔹 Appel de la fonction : Encodage avec Hamming (7,4)

- **Protège les données** contre les erreurs en ajoutant des bits de parité.
- **Affiche le message encodé**.


In [ ]:
# Encodage avec le code de Hamming
encoded_bits = hamming_encode(bits)
print(f"Message encodé avec Hamming : {encoded_bits}")


## 🔹 Appel de la fonction : Modulation FSK

- **Transforme les bits en signal audio** à haute fréquence.


In [ ]:
# Modulation FSK
signal = fsk_modulation(encoded_bits)


## 🔹 Normalisation et Sauvegarde du Signal

- **Normalise** le signal pour éviter la saturation.
- **Convertit le signal** en format audio 16 bits.
- **Enregistre le signal** dans un fichier `.wav`.


In [ ]:
# Normalisation du signal pour l'enregistrement
signal = signal / np.max(np.abs(signal))  # Normalisation pour éviter la saturation
signal_int16 = np.int16(signal * 32767)

# Sauvegarde dans un fichier WAV
try:
    wav.write(filename, fs, signal_int16)
    print(f"✅ Signal FSK enregistré dans {filename}")
except Exception as e:
    print(f"Erreur lors de l'enregistrement du fichier WAV : {e}")


## 🔹 Lecture du Signal Audio

- **Joue le signal audio modulé** pour permettre d'entendre la transmission des données.


In [ ]:
# Lecture du signal audio
try:
    sd.play(signal, samplerate=fs)
    sd.wait()
    print("🔊 Lecture terminée.")
except Exception as e:
    print(f"Erreur lors de la lecture audio : {e}")


# 📥 **Partie 2 : Récepteur (Démodulation et Décodage du Signal FSK)**


## 📌 Importation des Bibliothèques

Ces bibliothèques sont nécessaires pour :  
- **NumPy** (`numpy`) : manipulation des données numériques.  
- **SciPy** (`scipy.io.wavfile`) : lecture et traitement du fichier audio `.wav`.  
- **SciPy** (`scipy.signal`) : filtrage du signal audio.  
- **OS** (`os`) : gestion des fichiers et du chemin d'accès.


In [ ]:
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal as sp_signal
import os


## 🎯 Définition des Paramètres

Ces paramètres **doivent correspondre** à ceux de l'émetteur pour garantir la bonne réception du signal.  
- `fs` : fréquence d'échantillonnage du signal audio.  
- `bit_rate` : vitesse de transmission en bits/seconde.  
- `f0` et `f1` : fréquences associées aux bits `0` et `1`.  
- `bit_duration` : durée de transmission d'un bit.


In [ ]:
# Paramètres de démodulation (identiques à ceux de l'émetteur)
fs = 44100  # Fréquence d'échantillonnage (Hz)
bit_rate = 100  # Débit binaire (bits/sec)
f0 = 17000  # Fréquence du bit '0' (Hz)
f1 = 19000  # Fréquence du bit '1' (Hz)
bit_duration = 1 / bit_rate  # Durée d'un bit (s)


## 📁 Définition du Chemin du Fichier

Le fichier `.wav` doit être localisé au même endroit que celui utilisé par l'émetteur pour assurer la récupération correcte des données.


In [ ]:
# Définition du chemin du fichier audio
input_dir = r"C:\Users\CORENTIN CARTALLIER\Desktop\CPI A1\PROJET 3\Avec importation haute fréquence"
filename = os.path.join(input_dir, "fsk_transmission.wav")


## 🔹 Filtrage Passe-Bande

Cette fonction filtre le signal audio pour **conserver uniquement les fréquences utilisées** pour la transmission FSK.


In [ ]:
def bandpass_filter(data, fs, lowcut=16500, highcut=19500, order=4):
    """Applique un filtre passe-bande Butterworth pour extraire le signal utile."""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = sp_signal.butter(order, [low, high], btype='band')
    return sp_signal.filtfilt(b, a, data)


## 🔹 Démodulation FSK

La démodulation repose sur une **analyse spectrale** pour détecter la fréquence dominante et en déduire les bits `0` et `1`.


In [ ]:
def demodulate_fsk(audio_signal):
    """Démodule le signal FSK pour extraire les bits de données."""
    bits = ""
    samples_per_bit = int(bit_duration * fs)

    for i in range(0, len(audio_signal), samples_per_bit):
        segment = audio_signal[i:i + samples_per_bit]

        if len(segment) < samples_per_bit:
            continue

        # Analyse spectrale (FFT) pour détecter la fréquence dominante
        fft_result = np.fft.fft(segment)
        freqs = np.fft.fftfreq(len(segment), d=1/fs)
        magnitude = np.abs(fft_result)

        # Comparaison de l’énergie des fréquences `f0` et `f1`
        f0_energy = np.sum(magnitude[(freqs >= f0 - 50) & (freqs <= f0 + 50)])
        f1_energy = np.sum(magnitude[(freqs >= f1 - 50) & (freqs <= f1 + 50)])

        bit = '1' if f1_energy > f0_energy else '0'
        bits += bit

    return bits


## 🔹 Décodage Hamming (7,4)

Cette fonction **corrige d'éventuelles erreurs de transmission** en détectant les erreurs via le **syndrome** et en appliquant une correction.


In [ ]:
def hamming_decode(encoded):
    """Décode les bits avec un code de Hamming (7,4) pour corriger d'éventuelles erreurs."""
    n = len(encoded)
    decoded = []

    for i in range(0, n, 7):
        code = encoded[i:i+7]
        p1, p2, d1, p3, d2, d3, d4 = code
        new_p1 = int(d1) ^ int(d2) ^ int(d3)
        new_p2 = int(d1) ^ int(d2) ^ int(d4)
        new_p3 = int(d2) ^ int(d3) ^ int(d4)
        syndrome = f"{int(p1) ^ new_p1}{int(p2) ^ new_p2}{int(p3) ^ new_p3}"

        if syndrome != "000":
            error_pos = int(syndrome, 2) - 1
            code = code[:error_pos] + str(int(code[error_pos]) ^ 1) + code[error_pos+1:]

        decoded.append(f"{code[2]}{code[4]}{code[5]}{code[6]}")

    return ''.join(decoded)


## 🔹 Conversion des Bits en Texte

Cette fonction **convertit les bits reçus** en caractères ASCII pour **retrouver le message original**.


In [ ]:
def bits_to_text(bits):
    """Convertit une suite de bits en texte ASCII."""
    chars = [chr(int(bits[i:i+8], 2)) for i in range(0, len(bits), 8)]
    return ''.join(chars)


## 🔹 Exécution de la Réception du Signal

1️⃣ **Vérifie si le fichier `.wav` existe** et charge les données.  
2️⃣ **Normalise** le signal audio pour éviter les erreurs numériques.  
3️⃣ **Applique un filtre passe-bande** pour isoler les fréquences `f0` et `f1`.  
4️⃣ **Démodule le signal FSK** pour retrouver les bits transmis.  
5️⃣ **Corrige les erreurs avec Hamming (7,4)** pour fiabiliser les données.  
6️⃣ **Convertit les bits en texte ASCII** et affiche le message original.  


In [ ]:
# Vérification de l'existence du fichier
if not os.path.exists(filename):
    print(f"❌ Erreur : le fichier {filename} n'existe pas !")
    exit()

# Étape 1 : Chargement du fichier audio
print(f"📂 Chargement du fichier : {filename}")
fs, audio_signal = wav.read(filename)

# Étape 2 : Normalisation du signal
audio_signal = audio_signal.astype(np.float32) / 32767  # Conversion en float

# Étape 3 : Application du filtre passe-bande
audio_signal = bandpass_filter(audio_signal, fs)

# Étape 4 : Démodulation FSK
decoded_bits = demodulate_fsk(audio_signal)
print(f"🔢 Bits reçus : {decoded_bits}")

# Étape 5 : Décodage avec le code de Hamming
corrected_bits = hamming_decode(decoded_bits)
print(f"🔢 Bits corrigés : {corrected_bits}")

# Étape 6 : Conversion en texte
decoded_message = bits_to_text(corrected_bits)
print(f"💬 Message reçu : {decoded_message}")


# 🔚 Conclusion

Ce projet met en œuvre un **système de transmission et réception de données via un signal audio modulé en FSK**.
L’émetteur convertit un message texte en signal sonore haute fréquence, tandis que le récepteur **démodule** ce signal pour retrouver les données envoyées.

Grâce au **codage de Hamming (7,4)**, les erreurs de transmission peuvent être détectées et corrigées, rendant le système plus fiable.  
Ce type de transmission est utile dans des applications où l'utilisation d'ondes sonores pour échanger des données peut être pertinente, comme en **communications sous-marines, en IoT ou en transmission discrète d'informations**.

🚀 **Ce projet illustre donc un principe fondamental des télécommunications numériques !**


# 🔚 Conclusion

Ce projet montre comment on peut **transmettre un message sous forme de son** et le **récupérer après transmission**.  
L'émetteur transforme un texte en signal audio, puis le récepteur **démodule et corrige** les erreurs pour retrouver le message original.

